In [1]:
import pandas as pd
import numpy as np
import re


In [9]:

path = "data/실물지표/소매판매액지수raw.csv"

# 1) 로드 (KOSIS/통계청 CSV는 cp949인 경우가 많음)
df = pd.read_csv(path, encoding="cp949")

# 2) 설명행(불변지수 등) 제거: 숫자로 변환 안 되는 행이 보통 1줄 끼어있음
#    (예: 업종별='업종별', 각 월 값='불변지수')
df = df[df["업종별"].ne("업종별")].copy()

# 3) 원하는 시계열(행) 선택
target = "음식점 포함 소매판매액지수"
row = df[df["업종별"] == target].copy()
if row.empty:
    raise ValueError(f"'{target}' 행을 못 찾았어. df['업종별'].unique()로 실제 이름 확인해줘.")

# 4) wide -> long
long = row.melt(id_vars=["업종별"], var_name="ym_raw", value_name="value")

# 5) '2025.11 p)' 같은 꼬리표 제거해서 YYYY.MM만 추출
long["year_month"] = long["ym_raw"].astype(str).str.extract(r"(\d{4}\.\d{2})", expand=False)

# year_month 못 뽑은 컬럼(비정상 컬럼) 제거
long = long.dropna(subset=["year_month"]).copy()

# 6) 값 숫자화
long["value"] = pd.to_numeric(long["value"], errors="coerce")
long = long.dropna(subset=["value"]).copy()

# 7) 중복 월 처리(1월이 2번 있는 문제): 같은 year_month면 평균으로 1개로 합치기
long = (long.groupby("year_month", as_index=False)["value"].mean()
            .sort_values("year_month"))

# 8) datetime 인덱스 만들기
long["date"] = pd.PeriodIndex(long["year_month"], freq="M").to_timestamp()
long = long.sort_values("date").reset_index(drop=True)

# 9) YoY(%) 계산 + z-score
long["yoy"] = long["value"].pct_change(12) * 100
long = long.dropna(subset=["yoy"]).copy()

long["yoy_z"] = (long["yoy"] - long["yoy"].mean()) / long["yoy"].std(ddof=0)

# 결과
long[["date", "value", "yoy", "yoy_z"]].head(15)

long.to_csv('data/실물지표/소매판매액지수.csv')
long

,year_month,value,date,yoy,yoy_z
12,2011.01,87.5,2011-01-01,8.695652,2.228980
13,2011.02,77.5,2011-02-01,-0.768246,-0.691519
14,2011.03,86.4,2011-03-01,4.221954,0.848425
15,2011.04,85.9,2011-04-01,5.140759,1.131962
16,2011.05,90.3,2011-05-01,5.985915,1.392772
...,...,...,...,...,...
187,2025.08,102.3,2025-08-01,-0.097656,-0.484580
188,2025.09,106.9,2025-09-01,2.394636,0.284526
189,2025.10,105.1,2025-10-01,0.095238,-0.425054
190,2025.11,106.2,2025-11-01,0.663507,-0.249689


In [11]:


path = "data/실물지표/신용카드사용액raw.csv"

# 1) 로드 (국내 통계 CSV는 cp949가 흔함)
df = pd.read_csv(path, encoding="cp949")

# 2) (선택) 특정 유형만 쓰고 싶으면 여기서 고르기
#    파일상으론 '합계' 1개 행이지만, 행이 여러개일 수도 있어서 안전하게 처리
target = "합계"
if target in df["금융기관 유형코드"].astype(str).values:
    df = df[df["금융기관 유형코드"].astype(str) == target].copy()
else:
    # 합계가 없으면 첫 행 사용
    df = df.iloc[[0]].copy()

# 3) wide -> long (YYYY/MM 컬럼만 melt)
id_cols = ["금융기관 유형코드", "단위"]
value_cols = [c for c in df.columns if re.fullmatch(r"\d{4}/\d{2}", str(c))]

long = df.melt(id_vars=id_cols, value_vars=value_cols,
               var_name="year_month", value_name="value")

# 4) 숫자화 (콤마/공백 제거 후 float)
long["value"] = (long["value"].astype(str)
                            .str.replace(",", "", regex=False)
                            .str.strip())
long["value"] = pd.to_numeric(long["value"], errors="coerce")
long = long.dropna(subset=["value"]).copy()

# 5) 날짜 만들기
long["date"] = pd.to_datetime(long["year_month"], format="%Y/%m")

# 6) 혹시 같은 월이 중복이면 평균으로 1개로 통합
long = (long.groupby("date", as_index=False)["value"].mean()
            .sort_values("date")
            .reset_index(drop=True))

# 7) YoY(전년동월대비, %) -> z-score
long["yoy"] = long["value"].pct_change(12) * 100
long = long.dropna(subset=["yoy"]).copy()

# ddof=0(모표준편차)로 고정해서 지표들끼리 일관되게
long["yoy_z"] = (long["yoy"] - long["yoy"].mean()) / long["yoy"].std(ddof=0)

# 결과 확인
long[["date", "value", "yoy", "yoy_z"]].tail(12)
long.to_csv('data/실물지표/신용카드사용액.csv')
long

,date,value,yoy,yoy_z
12,2011-01-01,44348384.0,18.916531,2.736683
13,2011-02-01,39286019.0,10.060224,0.914017
14,2011-03-01,45802472.0,1.806173,-0.784702
15,2011-04-01,43365817.0,9.489950,0.796652
16,2011-05-01,47226380.0,16.356493,2.209816
...,...,...,...,...
186,2025-07-01,94958632.0,5.445655,-0.035681
187,2025-08-01,91516719.0,3.409900,-0.454648
188,2025-09-01,95364571.0,7.032161,0.290829
189,2025-10-01,92592210.0,1.679985,-0.810672


In [12]:

path = "data/실물지표/전산업생산지수raw.csv"

# 1) 로드
df = pd.read_csv(path, encoding="cp949")

# 2) 설명행 제거 (숫자 아닌 행 제거)
# 첫 컬럼이 업종/지수명일 가능성 높음
first_col = df.columns[0]
df = df[df[first_col] != first_col].copy()

# 3) 전산업 선택
# 실제 이름 확인 필요 (예: '전산업', '전산업지수', '전산업생산지수')
print(df[first_col].unique())  # 한번 확인용

target = "전산업"
if target in df[first_col].astype(str).values:
    df = df[df[first_col].astype(str) == target].copy()
else:
    df = df.iloc[[0]].copy()  # 없으면 첫 행 사용

# 4) YYYY/MM 또는 YYYY.MM 형태 컬럼만 선택
value_cols = [c for c in df.columns if re.search(r"\d{4}[/\.]\d{2}", str(c))]

long = df.melt(id_vars=[first_col], value_vars=value_cols,
               var_name="year_month_raw", value_name="value")

# 5) 연월 정리 (YYYY.MM 형태 추출)
long["year_month"] = long["year_month_raw"].astype(str)\
                                           .str.extract(r"(\d{4}[/\.]\d{2})")[0]

# 6) 숫자 변환
long["value"] = (long["value"].astype(str)
                            .str.replace(",", "", regex=False)
                            .str.strip())
long["value"] = pd.to_numeric(long["value"], errors="coerce")
long = long.dropna(subset=["value", "year_month"]).copy()

# 7) 날짜 변환
long["year_month"] = long["year_month"].str.replace("/", ".", regex=False)
long["date"] = pd.to_datetime(long["year_month"], format="%Y.%m")

# 8) 중복 월 처리
long = (long.groupby("date", as_index=False)["value"].mean()
            .sort_values("date")
            .reset_index(drop=True))

# 9) YoY 계산
long["yoy"] = long["value"].pct_change(12) * 100
long = long.dropna(subset=["yoy"]).copy()

# 10) z-score 계산
long["yoy_z"] = (long["yoy"] - long["yoy"].mean()) / long["yoy"].std(ddof=0)

# 결과 확인
long[["date", "value", "yoy", "yoy_z"]].tail(12)
long.to_csv('data/실물지표/전산업생산지수.csv')

['전산업생산지수']


In [18]:
card_df

,Unnamed: 0,date,value,yoy,yoy_z
0,12,2011-01-01,44348384.0,18.916531,2.736683
1,13,2011-02-01,39286019.0,10.060224,0.914017
2,14,2011-03-01,45802472.0,1.806173,-0.784702
3,15,2011-04-01,43365817.0,9.489950,0.796652
4,16,2011-05-01,47226380.0,16.356493,2.209816
...,...,...,...,...,...
174,186,2025-07-01,94958632.0,5.445655,-0.035681
175,187,2025-08-01,91516719.0,3.409900,-0.454648
176,188,2025-09-01,95364571.0,7.032161,0.290829
177,189,2025-10-01,92592210.0,1.679985,-0.810672


In [22]:
retail_df = pd.read_csv('data/실물지표/소매판매액지수.csv')
card_df = pd.read_csv('data/실물지표/신용카드사용액.csv')
ip_df = pd.read_csv('data/실물지표/전산업생산지수.csv')

retail_df=retail_df[['date','yoy_z']].rename(columns={'yoy_z': 'retail'})
card_df=card_df[['date','yoy_z']].rename(columns={'yoy_z': 'card'})
ip_df=ip_df[['date','yoy_z']].rename(columns={'yoy_z': 'ip'})


real_df = retail_df.merge(card_df, on="date", how="inner")\
                    .merge(ip_df, on="date", how="inner")

real_df["real_index"] = real_df[["retail",
                                  "card",
                                  "ip"]].mean(axis=1)

real_df.to_csv('data/실물지표/실물지표계산.csv')